In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
file_path=r'E:\Machine Learning\team-learning-master'
data1=r'\端午粽子数据.csv'
data2=r'\摩拜单车数据.csv'
data3=r'\墨尔本温度数据.csv'

## 一.端午节的淘宝粽子交易
问题

(1) 请删除最后一列为缺失值的行，并求所有在杭州发货的商品单价均值。

(2) 商品标题带有“嘉兴”但发货地却不在嘉兴的商品有多少条记录？

(3) 请按照分位数将价格分为“高、较高、中、较低、低”5 个类别，再将

类别结果插入到标题一列之后，最后对类别列进行降序排序。

(4) 付款人数一栏有缺失值吗？若有则请利用上一问的分类结果对这些缺失
值进行合理估计并填充。

(5) 请将数据后四列合并为如下格式的 Series：商品发货地为 ××，店铺为
××，共计 ×× 人付款，单价为 ××。

(6) 请将上一问中的结果恢复成原来的四列。

In [4]:
df1=pd.read_csv(file_path+data1)
df1.head()

,标题,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,129,6人付款,五芳斋官方旗舰店,浙江 嘉兴
1,北京稻香村端午粽子手工豆沙粽220g*2袋散装豆沙粽香甜软糯豆沙粽,44,8人付款,天猫超市,上海
2,五芳斋粽子礼盒装鲜肉咸蛋黄大肉粽嘉兴豆沙甜粽端午团购散装礼品,89.9,100万+人付款,五芳斋官方旗舰店,浙江 嘉兴
3,稻香私房鲜肉粽蛋黄肉粽嘉兴粽子咸鸭蛋礼盒装端午节送礼特产团购,138,1936人付款,稻香村食品旗舰店,北京
4,嘉兴粽子 蛋黄鲜肉粽新鲜大肉粽早餐散装团购浙江特产蜜枣多口味,3.8,9500+人付款,城城喂食猫,浙江 嘉兴


第一题

In [15]:
df1.dropna(axis=0, how='any', subset=['发货地址 '],inplace=True)

In [16]:
len(df1)#4403，删除了3个含缺失值的行，剩余4400行

4400

In [19]:
df1.columns#注意空格

Index(['标题', ' 价格', '付款人数', '店铺', '发货地址 '], dtype='object')

In [22]:
df1.info()#真是很神奇的数据格式，全为object

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4400 entries, 0 to 4402
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   标题      4400 non-null   object
 1    价格     4400 non-null   object
 2   付款人数    4329 non-null   object
 3   店铺      4400 non-null   object
 4   发货地址    4400 non-null   object
dtypes: object(5)
memory usage: 206.2+ KB


In [34]:
#使用正则匹配，将不符合条件的数据排除，只计算符合条件数据
df1[df1['发货地址 ']=='浙江 杭州'][' 价格'][(df1[' 价格'].str.replace(r'\d+\.?\d+','True')=='True')].astype('float').mean()

81.50651685393247

In [35]:
#看一下不符合条件的数据，可以看到只有4376，这里稍微修改一下
df1[df1['发货地址 ']=='浙江 杭州'][~(df1[' 价格'].str.replace(r'\d+\.?\d+','True')=='True')]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,标题,价格,付款人数,店铺,发货地址
4146,美珍妈妈粽子甜粽 红豆粽豆沙粽红豆蜜枣粽白米粽多口味嘉兴粽子,9,611人付款,美珍粽子,浙江 杭州
4376,东方贺礼端午节礼盒嘉兴粽子高汤蛋黄鲜大肉粽特产现已支持定制,45,17人付款,缘来有礼,浙江 杭州


In [33]:
df1.loc[4376,' 价格']=45

In [38]:
df1[df1['发货地址 ']=='浙江 杭州'][' 价格'].astype('float').mean()

80.90088888888877

第二题

In [43]:
df1[df1['发货地址 ']!='浙江 嘉兴'][(df1['标题'].str.replace('.*嘉兴.*','True')=='True')]['标题'].count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


1032

第三题

In [45]:
df1[' 价格']=df1[' 价格'].astype('float')
df1[' 价格'].mean()

73.65447272727197

In [61]:
cutlist=[df1[' 价格'].quantile(x) for x in np.linspace(0,1,6)]
label=['高','较高','中','较低','低']
label.reverse()
df1['价格类型']=pd.cut(df1[' 价格'],cutlist,labels=label)
col=['标题','价格类型',' 价格','付款人数', '店铺', '发货地址 ']
df1=df1[col]

,标题,价格类型,价格,付款人数,店铺,发货地址
3136,包邮端午节大家乐大黄米粽子蜜枣红豆沙榴莲蛋黄肉梅菜肉馅团订,低,19.0,95人付款,大家乐网上专卖店,山东 烟台
3698,广州酒家肇庆裹蒸粽碱水粽广东粽子利口福蛋黄肉粽豆沙甜粽团购,低,19.8,30人付款,广东东邮食品专营店,广东 东莞
3700,贞妃贞丰粽子 贵州特产 鲜肉板栗排骨灰粽超大粽咸味散装农家手工,低,16.8,48人付款,锋行食品专营店,贵州 黔西南
3155,嘉兴蛋黄鲜肉粽红豆粽蜜枣粽排骨粽早餐鲜肉粽端午节团购批发粽子,低,20.9,1.5万+人付款,健谷鲜旗舰店,浙江 温州
3706,「刘叔叔手作」独家秘制新鲜蛋黄梅菜瘦肉手工长粽子长条 鲜肉粽,低,13.0,234人付款,轻描淡写68,重庆
...,...,...,...,...,...,...
1068,【天猫福宝专享】五芳斋丰年五芳粽子礼盒装蛋黄肉粽大肉粽子嘉兴,高,199.0,88人付款,五芳斋官方旗舰店,浙江 嘉兴
1096,泉州特产蓝氏鲜肉粽子2000克早餐东街钟楼闽南烧肉粽端午团购礼盒,高,158.0,22人付款,蓝氏钟楼旗舰店,福建 泉州
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129.0,6人付款,五芳斋官方旗舰店,浙江 嘉兴
1548,久知味醇香鲜肉粽135g 真空粽嘉兴特产粽子速食早餐,NaN,2.8,NaN,天猫会员店,上海


In [63]:
df1.sort_values(by='价格类型',ascending=False).head()

,标题,价格类型,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129.0,6人付款,五芳斋官方旗舰店,浙江 嘉兴
660,五芳斋粽子百年五芳礼品粽大肉粽豆沙粽嘉兴粽子礼盒装 批发团购,高,328.0,20人付款,伟弘食品专营店,上海
1945,好利来礼盒粽子好礼香棕顺丰包邮沈阳发货员工福利采购团购优惠,高,109.0,11人付款,木宝很开心,辽宁 沈阳
1197,诸老大粽子诸事欢禧礼盒嘉兴特产蛋黄黑猪肉粽咸蛋端午送礼团购,高,180.0,20人付款,诸老大旗舰店,浙江 嘉兴
3171,重庆莎莲娜四川粽子端午节礼盒装团购2100g送礼领导送客户多口味,高,228.0,52人付款,melonsha88,重庆


第四题

In [57]:
#取价格类型为相同的组中付款人数做均值填充
df1.groupby('价格类型').apply(x.fillna())

In [58]:
df1.head()

,标题,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,129.0,6人付款,五芳斋官方旗舰店,浙江 嘉兴
1,北京稻香村端午粽子手工豆沙粽220g*2袋散装豆沙粽香甜软糯豆沙粽,44.0,8人付款,天猫超市,上海
2,五芳斋粽子礼盒装鲜肉咸蛋黄大肉粽嘉兴豆沙甜粽端午团购散装礼品,89.9,100万+人付款,五芳斋官方旗舰店,浙江 嘉兴
3,稻香私房鲜肉粽蛋黄肉粽嘉兴粽子咸鸭蛋礼盒装端午节送礼特产团购,138.0,1936人付款,稻香村食品旗舰店,北京
4,嘉兴粽子 蛋黄鲜肉粽新鲜大肉粽早餐散装团购浙江特产蜜枣多口味,3.8,9500+人付款,城城喂食猫,浙江 嘉兴
